In [32]:
import nltk
import pandas as pd
import re
import time
import multiprocessing
import io
# import gensim
import numpy as np
import matplotlib.pyplot as plt
import pickle
# import keras
from sklearn.model_selection import KFold # import KFold
from sklearn.model_selection import StratifiedKFold
# from tensorflow import keras
# from keras.models import Sequential
# from tensorflow.keras.optimizers import Adam
# from keras.callbacks import ModelCheckpoint
# from keras.preprocessing.text import Tokenizer
# from keras.models import Sequential
# from keras.layers import Embedding
# from keras.layers import Dense, Activation, Embedding, LSTM, Bidirectional, Dropout, GRU
# from keras import regularizers
# from tensorflow.keras.utils import to_categorical
# from keras.models import load_model
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn import metrics
from nltk.tokenize import TweetTokenizer
from collections import defaultdict
from datetime import timedelta
# from gensim.models import word2vec
from sklearn.metrics import classification_report
from sklearn.metrics import f1_score, precision_score, recall_score, confusion_matrix, accuracy_score
from sklearn.feature_extraction.text import CountVectorizer
from varname import nameof

from nltk.tokenize import word_tokenize
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory

#2.Load Data

In [120]:
#simpan path dataset
path_data_aspek = "data_scraping_kota_TGL.csv"
#read dataset
data_aspek = pd.read_csv(path_data_aspek, sep=",", header=[0], encoding="UTF-8")
data_aspek.head()

,Unnamed: 0,date,username,tweet,wisata/hiburan,pendidikan,fasilitas/layanan publik,kuliner
0,0,09/12/2021 23:11,BsiWahyudi,Wisuda Universitas BSI Kampus Kabupaten Banyum...,-1,1,-1,-1
1,1,09/12/2021 20:44,PPPLarangan,(03/12/2021) Mengikuti kegiatan bersih pantai...,-1,-1,1,-1
2,2,09/12/2021 19:46,AdvFeryAjah,"@infoplntegal Selamat malam , mohon di tindak ...",-1,-1,0,-1
3,3,09/12/2021 19:27,tegalfess,"[TF] info banjir daerah tegal kota dong lurr, ...",-1,-1,-1,1
4,4,09/12/2021 14:58,detikcom,Pemasangan portal di jalan masuk Alun-alun Kot...,-1,-1,0,-1


#3.Implementasi Preprocessing

In [35]:
#menyimpan tweet. (tipe data series pandas)
data_content = data_aspek['tweet']

##3.1. Casefolding

In [36]:
# casefolding
data_casefolding = data_content.str.lower()
data_casefolding.head()

0    wisuda universitas bsi kampus kabupaten banyum...
1    (03/12/2021)  mengikuti kegiatan bersih pantai...
2    @infoplntegal selamat malam , mohon di tindak ...
3    [tf] info banjir daerah tegal kota dong lurr, ...
4    pemasangan portal di jalan masuk alun-alun kot...
Name: tweet, dtype: object

##3.2. Filtering

In [37]:
#filtering

#url
filtering_url = [re.sub(r'''(?i)\b((?:https?://|www\d{0,3}[.]|[a-z0-9.\-]+[.][a-z]{2,4}/)(?:[^\s()<>]+|\(([^\s()<>]+|(\([^\s()<>]+\)))*\))+(?:\(([^\s()<>]+|(\([^\s()<>]+\)))*\)|[^\s`!()\[\]{};:'".,<>?«»“”‘’]))''', " ", tweet) for tweet in data_casefolding]
#cont
filtering_cont = [re.sub(r'\(cont\)'," ", tweet)for tweet in filtering_url]
#punctuatuion
filtering_punctuation = [re.sub('[!"”#$%&’()*+,-./:;<=>?@[\]^_`{|}~]', ' ', tweet) for tweet in filtering_cont]  #hapus simbol'[!#?,.:";@()-_/\']'
#  hapus #tagger
filtering_tagger = [re.sub(r'#([^\s]+)', '', tweet) for tweet in filtering_punctuation]
#numeric
filtering_numeric = [re.sub(r'\d+', ' ', tweet) for tweet in filtering_tagger]

# # filtering RT , @ dan #
# fungsi_clen_rt = lambda x: re.compile('\#').sub('', re.compile('rt @').sub('@', x, count=1).strip())
# clean = [fungsi_clen_rt for tweet in filtering_numeric]

data_filtering = pd.Series(filtering_numeric)


## 3.3. Tokenisasi

In [123]:
# #tokenize
tknzr = TweetTokenizer()
data_tokenize = [tknzr.tokenize(tweet) for tweet in data_filtering]
data_tokenize

[['wisuda',
  'universitas',
  'bsi',
  'kampus',
  'kabupaten',
  'banyumas',
  'ke',
  'amp',
  'kampus',
  'kota',
  'tegal',
  'ke',
  'java',
  'heritage',
  'hotel',
  'purwokerto'],
 ['mengikuti',
  'kegiatan',
  'bersih',
  'pantai',
  'yang',
  'diselenggarakan',
  'lanal',
  'tegal',
  'di',
  'pantai',
  'alam',
  'indah',
  'kota',
  'tegal',
  'kegiatan',
  'ini',
  'diikuti',
  'oleh',
  'berbagai',
  'instansi',
  'terkait',
  'di',
  'kota',
  'tegal',
  'dan',
  'sekitarnya',
  'dkpjateng',
  'djpt',
  'kkp'],
 ['infoplntegal',
  'selamat',
  'malam',
  'mohon',
  'di',
  'tindak',
  'lanjuti',
  'di',
  'daerah',
  'saya',
  'dari',
  'jam',
  'wib',
  'mati',
  'listrik',
  'sampai',
  'sekarang',
  'belum',
  'menyala',
  'alamat',
  'di',
  'jl',
  'drcipto',
  'mangunkusumo',
  'kaligansa',
  'kecamatan',
  'margadana',
  'kota',
  'tegal'],
 ['tf',
  'info',
  'banjir',
  'daerah',
  'tegal',
  'kota',
  'dong',
  'lurr',
  'kayong',
  'pgn',
  'seblak',
  'kie',

##3.4. Konversi Slangword

In [39]:
#slang word
path_dataslang = open("kamus kata baku-clear (1).csv")
dataslang = pd.read_csv(path_dataslang, encoding = 'utf-8', header=None, sep=";")

def replaceSlang(word):
  if word in list(dataslang[0]):
    indexslang = list(dataslang[0]).index(word)
    return dataslang[1][indexslang]
  else:
    return word

data_formal = []
for data in data_tokenize:
  data_clean = [replaceSlang(word) for word in data]
  data_formal.append(data_clean)
len_data_formal = len(data_formal)
# print(data_formal)
# len_data_formal

In [40]:
dataslang

,0,1
0,tlg,tolong
1,dwpan,depan
2,dwngan,dengan
3,byk,banyak
4,smoga,semoga
...,...,...
5437,politic,politik
5438,cont,
5439,rt,
5440,cc,NaN


## 3.5. Stopword

In [41]:
nltk.download('stopwords')
default_stop_words = nltk.corpus.stopwords.words('indonesian')
stopwords = set(default_stop_words)

def removeStopWords(line, stopwords):
  words = []
  for word in line:  
    word=str(word)
    word = word.strip()
    if word not in stopwords and word != "" and word != "&":
      words.append(word)

  return words
reviews = [removeStopWords(line,stopwords) for line in data_formal]

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\novit\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [124]:
reviews

[['wisuda',
  'universitas',
  'bsi',
  'kampus',
  'kabupaten',
  'banyumas',
  'amp',
  'kampus',
  'kota',
  'tegal',
  'java',
  'heritage',
  'hotel',
  'purwokerto'],
 ['mengikuti',
  'kegiatan',
  'bersih',
  'pantai',
  'diselenggarakan',
  'lanal',
  'tegal',
  'pantai',
  'alam',
  'indah',
  'kota',
  'tegal',
  'kegiatan',
  'diikuti',
  'instansi',
  'terkait',
  'kota',
  'tegal',
  'dkpjateng',
  'djpt',
  'kkp'],
 ['infoplntegal',
  'selamat',
  'malam',
  'mohon',
  'tindak',
  'lanjuti',
  'daerah',
  'jam',
  'wib',
  'mati',
  'listrik',
  'menyala',
  'alamat',
  'jl',
  'drcipto',
  'mangunkusumo',
  'kaligansa',
  'kecamatan',
  'margadana',
  'kota',
  'tegal'],
 ['tf',
  'info',
  'banjir',
  'daerah',
  'tegal',
  'kota',
  'lurr',
  'kayong',
  'seblak',
  'kie',
  'makasii'],
 ['pemasangan',
  'portal',
  'jalan',
  'masuk',
  'alun',
  'alun',
  'kota',
  'tegal',
  'diprotes',
  'pedagang',
  'wali',
  'kota',
  'tegal',
  'deddy',
  'yon',
  'suriyono',
 

# 4.Implementasi Naive Bayes (Klasifikasi Aspek)

## 4.1.pembuatan vector kata

In [43]:
# pembuatan vector kata
vectorizer = TfidfVectorizer()
reviews2 = [" ".join(r) for r in reviews]
vektor_tfidf = vectorizer.fit_transform(reviews2)
vektor_tfidf = vektor_tfidf.toarray()
vektor_tfidf.shape

(380, 2023)

In [44]:
print(vektor_tfidf)
# print(vectorizer.get_feature_names())

[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


## Aspek Wisata

### Modeling

In [122]:
#Panggil Data target aspek (y)
label_aspek_1 = data_aspek['wisata/hiburan']
label_aspek_1.shape

(380,)

In [125]:
len(label_aspek_1)

380

In [91]:
X_train, X_test, y_train, y_test = train_test_split(vektor_tfidf, label_aspek_1, test_size=0.3)#, shuffle=True)
nama = "Model_Aspek_Wisata_Hiburan"
y_train = np.array(y_train)
y_test = np.array(y_test)

In [76]:
# contoh menggunakan RandomOversampler
from imblearn.over_sampling import RandomOverSampler
ros = RandomOverSampler(random_state=0)
X_resampled, y_resampled = ros.fit_resample(X_train, y_train)
from collections import Counter
print(sorted(Counter(y_resampled).items()))

[(-1, 227), (0, 227), (1, 227)]


In [81]:
from sklearn import metrics
clf = MultinomialNB()
clf.fit(X_resampled, y_resampled)
y_pred = clf.predict(X_resampled)
print("Akurasi : "+str(clf.score(X_resampled, y_resampled)))

print("Precission : "+str(metrics.precision_score(y_resampled, y_pred, labels=[-1,0,1], average='macro')))

Akurasi : 0.9706314243759178
Precission : 0.9721784110754578


In [82]:
# save model
# model_nvb = 'tfidf_sentimen_integritas_nvb.sav'
pickle.dump(clf, open('model/tfidf_{}_nvb.pkl'.format(nama), 'wb'))

### Pengujian

In [93]:
model = pickle.load(open('model/tfidf_'+str(nama)+'_nvb.pkl','rb'))
pred_y_ = model.predict(X_test)
label_test = y_test
# result = model.score(X_test, y_test)
print('\n'+'-'*45)
print('confusion matrix : ' + str(nama))
print('-'*45,'\n')
print(classification_report(label_test, pred_y_, labels=[-1, 0, 1]))


---------------------------------------------
confusion matrix : Model_Aspek_Wisata_Hiburan
--------------------------------------------- 

              precision    recall  f1-score   support

          -1       1.00      0.86      0.93        95
           0       0.12      1.00      0.22         1
           1       0.67      0.89      0.76        18

    accuracy                           0.87       114
   macro avg       0.60      0.92      0.64       114
weighted avg       0.94      0.87      0.89       114



## Aspek Pendidikan

In [94]:
label_aspek_2 = data_aspek['pendidikan']

In [99]:
X_train, X_test, y_train, y_test = train_test_split(vektor_tfidf, label_aspek_2, test_size=0.3)#, shuffle=True)
nama = "Model_Aspek_Wisata_Pendidikan"
y_train = np.array(y_train)
y_test = np.array(y_test)
# contoh menggunakan RandomOversampler
from imblearn.over_sampling import RandomOverSampler
ros = RandomOverSampler(random_state=0)
X_resampled, y_resampled = ros.fit_resample(X_train, y_train)
from collections import Counter
print(sorted(Counter(y_resampled).items()))
from sklearn import metrics
clf = MultinomialNB()
clf.fit(X_resampled, y_resampled)
y_pred = clf.predict(X_resampled)
print("Akurasi : "+str(clf.score(X_resampled, y_resampled)))
print("Precission : "+str(metrics.precision_score(y_resampled, y_pred, labels=[-1,0,1], average='macro')))

[(-1, 248), (0, 248), (1, 248)]
Akurasi : 0.9986559139784946
Precission : 0.998661311914324


In [100]:
# save model
# model_nvb = 'tfidf_sentimen_integritas_nvb.sav'
pickle.dump(clf, open('model/tfidf_{}_nvb.pkl'.format(nama), 'wb'))

In [101]:
model = pickle.load(open('model/tfidf_'+str(nama)+'_nvb.pkl','rb'))
pred_y_ = model.predict(X_test)
label_test = y_test
# result = model.score(X_test, y_test)
print('\n'+'-'*45)
print('confusion matrix : ' + str(nama))
print('-'*45,'\n')
print(classification_report(label_test, pred_y_, labels=[-1, 0, 1]))


---------------------------------------------
confusion matrix : Model_Aspek_Wisata_Pendidikan
--------------------------------------------- 

              precision    recall  f1-score   support

          -1       0.95      0.96      0.96       100
           0       0.25      1.00      0.40         1
           1       0.78      0.54      0.64        13

    accuracy                           0.91       114
   macro avg       0.66      0.83      0.66       114
weighted avg       0.92      0.91      0.91       114



## Aspek Fasilitas dan Layanan Public

In [102]:
label_aspek_3 = data_aspek['fasilitas/layanan publik']

In [103]:
X_train, X_test, y_train, y_test = train_test_split(vektor_tfidf, label_aspek_3, test_size=0.3)#, shuffle=True)
nama = "Model_Aspek_Wisata_Fasilitas_Layanan_Publik"
y_train = np.array(y_train)
y_test = np.array(y_test)
# contoh menggunakan RandomOversampler
from imblearn.over_sampling import RandomOverSampler
ros = RandomOverSampler(random_state=0)
X_resampled, y_resampled = ros.fit_resample(X_train, y_train)
from collections import Counter
print(sorted(Counter(y_resampled).items()))
from sklearn import metrics
clf = MultinomialNB()
clf.fit(X_resampled, y_resampled)
y_pred = clf.predict(X_resampled)
print("Akurasi : "+str(clf.score(X_resampled, y_resampled)))

print("Precission : "+str(metrics.precision_score(y_resampled, y_pred, labels=[-1,0,1], average='macro')))

[(-1, 114), (0, 114), (1, 114)]
Akurasi : 0.956140350877193
Precission : 0.9590224955053901


In [104]:
# save model
# model_nvb = 'tfidf_sentimen_integritas_nvb.sav'
pickle.dump(clf, open('model/tfidf_{}_nvb.pkl'.format(nama), 'wb'))
model = pickle.load(open('model/tfidf_'+str(nama)+'_nvb.pkl','rb'))
pred_y_ = model.predict(X_test)
label_test = y_test
# result = model.score(X_test, y_test)
print('\n'+'-'*45)
print('confusion matrix : ' + str(nama))
print('-'*45,'\n')
print(classification_report(label_test, pred_y_, labels=[-1, 0, 1]))
## Aspek Fasilitas dan Layanan Public


---------------------------------------------
confusion matrix : Model_Aspek_Wisata_Fasilitas_Layanan_Publik
--------------------------------------------- 

              precision    recall  f1-score   support

          -1       0.88      0.75      0.81        40
           0       0.63      0.73      0.68        33
           1       0.74      0.76      0.75        41

    accuracy                           0.75       114
   macro avg       0.75      0.74      0.74       114
weighted avg       0.76      0.75      0.75       114



## Aspek Kuliner

In [105]:
label_aspek_4 = data_aspek['kuliner']

In [118]:
X_train, X_test, y_train, y_test = train_test_split(vektor_tfidf, label_aspek_4, test_size=0.3)#, shuffle=True)
nama = "Model_Aspek_Wisata_Kuliner"
y_train = np.array(y_train)
y_test = np.array(y_test)
# contoh menggunakan RandomOversampler
from imblearn.over_sampling import RandomOverSampler
ros = RandomOverSampler(random_state=0)
X_resampled, y_resampled = ros.fit_resample(X_train, y_train)
from collections import Counter
print(sorted(Counter(y_resampled).items()))
from sklearn import metrics
clf = MultinomialNB()
clf.fit(X_resampled, y_resampled)
y_pred = clf.predict(X_resampled)
print("Akurasi : "+str(clf.score(X_resampled, y_resampled)))

print("Precission : "+str(metrics.precision_score(y_resampled, y_pred, labels=[-1,0,1], average='macro')))

[(-1, 219), (0, 219), (1, 219)]
Akurasi : 0.9832572298325722
Precission : 0.9839288077279345


In [119]:
# save model
# model_nvb = 'tfidf_sentimen_integritas_nvb.sav'
pickle.dump(clf, open('model/tfidf_{}_nvb.pkl'.format(nama), 'wb'))
model = pickle.load(open('model/tfidf_'+str(nama)+'_nvb.pkl','rb'))
pred_y_ = model.predict(X_test)
label_test = y_test
# result = model.score(X_test, y_test)
print('\n'+'-'*45)
print('confusion matrix : ' + str(nama))
print('-'*45,'\n')
print(classification_report(label_test, pred_y_, labels=[-1, 0, 1]))
## Aspek Fasilitas dan Layanan Public


---------------------------------------------
confusion matrix : Model_Aspek_Wisata_Kuliner
--------------------------------------------- 

              precision    recall  f1-score   support

          -1       1.00      0.88      0.94       102
           0       0.00      0.00      0.00         0
           1       0.55      0.92      0.69        12

    accuracy                           0.89       114
   macro avg       0.52      0.60      0.54       114
weighted avg       0.95      0.89      0.91       114



c:\Users\novit\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\novit\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\novit\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(r